This script will process single cell atlases for a sub cell type analysis.
1. load a single cell atlas; 
2. subset to a given cell type
3. perform simple QC and normalization 
4. save sc obj


In [1]:
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import os
from anndata.experimental.multi_files import AnnCollection
#import scanpy.external as sce


cell_type_oi= "Fib"

load atlases

In [2]:
def filter_data(adata, mt_percent=1, max_gene_count=2500):
    #filter genes and cells basd on min
    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=10)
    
    #add mt- gene as a threshold
    adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    
    #plot
    #sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
     #jitter=0.4, multi_panel=True)


    sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
    sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')
    
    #filter n gene count and mt percent
    adata = adata[adata.obs.n_genes_by_counts < max_gene_count, :]
    adata = adata[adata.obs.pct_counts_mt < mt_percent, :]
   

    return adata

def normalize_data(adata):
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, batch_key= "sample_id")
    sc.pl.highly_variable_genes(adata)
    #sc.pp.scale(adata, max_value=10)
    #sc.tl.pca(adata, svd_solver='arpack',  n_comps= 40)
    #sc.pl.umap(adata, color="sample_id")

    return adata



In [3]:
# all h5ads are stored 
#remote:
#sds_dir = "mnt/sds-hd/sd22b002/projects/reheat3/simplified/"

#local 
sds_dir = "/home/jan/R-projects/reheat2_pilot/data/simplified/"

files= os.listdir(sds_dir)
print(files)

dataset = [string.split("_")[0] for string in files]

print(dataset)
#



['Reichart2022_DCM.h5ad', 'Simonson2023_ICM.h5ad', 'Koenig2022_DCM.h5ad', 'Chaffin2022_DCM.h5ad']
['Reichart2022', 'Simonson2023', 'Koenig2022', 'Chaffin2022']


In [4]:
# Run all the steps and save object

for i in range(len(files)):
    print("Processing file:", files[i])
    scdat = sc.read_h5ad(sds_dir + files[i])

    #filter cells of interest
    scdat= scdat[scdat.obs.cell_type == cell_type_oi]
    
    # update the var names of this data set to gene symbols
    if files[i] == 'Reichart2022_DCM.h5ad':
        scdat.var_names= scdat.var["feature_name"]
    
    # add data set variable    
    scdat.obs['batch'] = dataset[i]

    #basic preprocessing:
    scdat= filter_data(scdat)
    scdat.layers["counts"] = scdat.X.copy()
    scdat = normalize_data(scdat)
    
    #write file: 
    print("Writing file:", files[i])
    scdat.write('/home/jan/R-projects/reheat2_pilot/data/celltype_data/Fibs/'+ dataset[i]+ "_fibs.h5ad" )
    



    




Processing file: Reichart2022_DCM.h5ad


/home/jan/programs/mambaforge/envs/scpy/lib/python3.11/site-packages/anndata/_core/anndata.py:949: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    ['MIR1302-2HG', 'FAM138A', 'OR4F5', 'RP11-34P13.7', 'RP11-34P13.8']

    Inferred to be: categorical

  names = self._prep_dim_index(names, "var")


KeyboardInterrupt: 